<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Competición: _Rock, paper, scissors, lizard, Spock_<a id="top"></a>
    
***

Bibliotecas usadas a lo largo del notebook

In [1]:
import enum
import itertools
import pickle
import typing
import urllib

import numpy as np

***

## Definición del juego

Elementos usados en el juego

In [2]:
# Acciones posibles
class Action(enum.Enum):
    """Cada una de las posibles figuras."""
    ROCK = '🪨'
    PAPER = '🧻'
    SCISSORS = '✂️'
    LIZARD = '🦎'
    SPOCK = '🖖'

# Recompensas asociadas al juego
MOVES_AND_REWARDS = {
    (Action.ROCK, Action.ROCK): 0, (Action.ROCK, Action.PAPER): -1,
    (Action.ROCK, Action.SCISSORS): 1, (Action.ROCK, Action.LIZARD): 1,
    (Action.ROCK, Action.SPOCK): -1,
    (Action.PAPER, Action.ROCK): 1, (Action.PAPER, Action.PAPER): 0,
    (Action.PAPER, Action.SCISSORS): -1, (Action.PAPER, Action.LIZARD): -1,
    (Action.PAPER, Action.SPOCK): 1,
    (Action.SCISSORS, Action.ROCK): -1, (Action.SCISSORS, Action.PAPER): 1,
    (Action.SCISSORS, Action.SCISSORS): 0, (Action.SCISSORS, Action.LIZARD): 1,
    (Action.SCISSORS, Action.SPOCK): -1,
    (Action.LIZARD, Action.ROCK): -1, (Action.LIZARD, Action.PAPER): 1,
    (Action.LIZARD, Action.SCISSORS): -1, (Action.LIZARD, Action.LIZARD): 0,
    (Action.LIZARD, Action.SPOCK): 1,
    (Action.SPOCK, Action.ROCK): 1, (Action.SPOCK, Action.PAPER): -1,
    (Action.SPOCK, Action.SCISSORS): 1, (Action.SPOCK, Action.LIZARD): -1,
    (Action.SPOCK, Action.SPOCK): 0,
}

Clase que define una partida entre dos jugadores

In [3]:
class Game:
    RENDER_MODE_HUMAN = 'human'
    
    def __init__(self, render_mode=None):
        self.render_mode = render_mode

    def play(self, p1_action, p2_action):
        result = MOVES_AND_REWARDS[(p1_action, p2_action)]
        if self.render_mode == 'human':
            self.render(p1_action, p2_action, result)
        return result
    
    @staticmethod
    def render(p1_action, p2_action, result):
        if result == 0:
            print(f'{p1_action.value} tie!')
        elif result == 1:
            print(f'{p1_action.value} beats {p2_action.value}')
        elif result == -1:
            print(f'{p2_action.value} beats {p1_action.value}')
        else:
            raise ValueError(f'{p1_action}, {p2_action}, {result}')

***

## Definición de comportamiento de un agente

Transiciones que realizan los agentes en el juego

In [4]:
class Transition(typing.NamedTuple):
    """Representa la transición de un estado al siguiente"""
    prev_state: int              # Estado origen de la transición
    next_state: int              # Estado destino de la transición
    action: Action               # Acción que provocó esta transición
    reward: typing.SupportsFloat # Recompensa obtenida

Clase que define el comportamiento de un agente

In [5]:
import abc

class Agent(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def __init__(self, name: str):
        """Inicializa el objeto.
        
        :param name: El nombre del agente.
        """
        self.name = name
        # START
        # END

    @abc.abstractmethod
    def decide(self, state:int) -> Action:
        """Decide la acción a llevar a cabo dado el estado actual.
        
        :param state: El estado en el que se encuentra el agente.
        :returns: La acción a llevar a cabo.
        """
        # START
        # END
    
    def update(self, transition: Transition):
        """Actualiza (si es necesario) el estado interno del agente.
        
        :param transition: La información de la transición efectuada.
        """
        pass
    
    def __str__(self):
        return self.name

***

## Agentes de los equipos

Como se definía en el enunciado, se permite cualquier implementación siempre y cuando:

1. Herede de la clase `Agent` suministrada.
1. El método `__init__` no admita ningún parámetro adicional.

### NeuralHive

In [6]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import random
from os import makedirs, cpu_count

from collections import deque

saved_path = "./Agente_NeuralHive/"
LisAct = list(Action)
dictAction = {k: v for v, k in enumerate(list(Action))}

# -------------------------- SETTING UP THE ENVIRONMENT --------------------------------------
# simple game, therefore we are not using the open gym custom set up
#---------------------------------------------------------------------------------------------
class RPSenv():
    def __init__ (self):
        self.action_space = Action		# integer representation of r/p/s/l/k
        self.seed = random.seed(42) 	# make it deterministic
        self.norm_mu = 0				# center point for guassian distribution
        self.norm_sigma = 4.0			# sigma for std distribution 
        self.seqIndex = 0				# index for pointing to the SEQ sequnce 
        self.p2Mode =  'SEQ'			# SEQ or PRNG or LFSR
        self.p2Count = [0, 0, 0, 0, 0] 		# player 2 win tie lost count
        self.p1Count = [0, 0, 0, 0, 0]		# player 1 win tie lost count
        self.window = 10					# window size for rate trending calc
        self.cumWinRate, self.cumTieRate, self.cumLostRate = None, None, None
        self.overallWinRate, self.overallTieRate, self.overallLostRate = 0, 0, 0
        self.cumWinCount, self.cumTieCount, self.cumLostCount = None, None, None
        self.winRateTrend, self.tieRateTrend, self.lostRateTrend = 0, 0, 0
        self.winRateMovingAvg, self.tieRateMovingAvg, self.lostRateMovingAvg = 0, 0, 0
        self.winRateBuf, self.tieRateBuf, self.lostRateBuf \
            = deque(maxlen=self.window), deque(maxlen=self.window), deque(maxlen=self.window)
        # put all the observation state in here; shape in Keras input format
        self.state = np.array([[ \
            None, None, None, \
            self.winRateTrend, self.tieRateTrend, self.lostRateTrend, \
            self.winRateMovingAvg, self.tieRateMovingAvg, self.lostRateMovingAvg \
            ]])  

    def reset(self):
        # reset all the state
        self.cumWinRate, self.cumTieRate, self.cumLostRate = 0, 0, 0
        self.cumWinCount, self.cumTieCount, self.cumLostCount = 0, 0, 0
        self.winRateTrend, self.tieRateTrend, self.lostRateTrend = 0, 0, 0
        self.winRateMovingAvg, self.tieRateMovingAvg, self.lostRateMovingAvg = 0, 0, 0
        return np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])

    def step(self, action, moveCount, stage):
        # This module generate one rock-paper-sessiors move based on the Mode selected.
        def genOneMove(self, mode, stage):
            import random
            def lfsr2(seed, taps, nbits):
                sr = seed
                while 1:
                    xor = 1
                    for t in taps:
                        if (sr & (1<<(t-1))) != 0:
                            xor ^= 1
                    sr = (xor << nbits-1) + (sr >> 1)
                    yield xor, sr
                    if sr == seed:
                        break
            if mode == 'PRNG':
                # change play strategy for player2 along the way 
                lowPlay  = 	{0:0, 1:1, 2:2, 3:4, 4:4} 			# key = stage number, value = r(0), p(1), s(2)
                mean1Play = {0:1, 1:2, 2:3, 3:3, 4:0} 			# key = stage number, value = r(0), p(1), s(2)
                mean2Play = {0:2, 1:3, 2:4, 3:0, 4:1} 			# key = stage number, value = r(0), p(1), s(2)
                mean3Play = {0:3, 1:4, 2:0, 3:1, 4:2}			# key = stage number, value = r(0), p(1), s(2)
                hiPlay = 	{0:4, 1:0, 2:1, 3:2, 4:3}			# key = stage number, value = r(0), p(1), s(2)
                # gen a random numbe from guassian & quantize it
                a = random.gauss(self.norm_mu, self.norm_sigma) 
                if a **2 < 1 and a > 0:  									# bettwen 1 and 0 is the paper move
                    play = mean2Play[stage]
                elif a > -1 and a < 0:										# else bettwen -1 and 0 is the rock move
                    play = mean3Play[stage]
                elif a < -1:												# else lower than  -1 is the sessiors move
                    play = lowPlay[stage] 
                elif a > 1 and a < 2:										# else bettwen 1 and 2 is the lizard move
                    play = mean1Play[stage]
                else:														# else higher than +2 is the spok move
                    play = hiPlay[stage]
                return LisAct[play]
            
            elif mode == 'SEQ':					# simple repeating pattern as 'random generator'
                dict = {'r':Action.ROCK, 'p':Action.PAPER, 's': Action.SCISSORS, 'l': Action.LIZARD, 'k': Action.SPOCK}
                seqlist = 'lrpprlsskkplskrsrlpprkspslkprspksplpsrkrspslpkrrlsspsklrpsklrpllskrslpkskk'			# the pattern sequence here
                self.seqIndex = (self.seqIndex + 1) % len(seqlist)
                return dict[seqlist[self.seqIndex]]

        # Lo deja en una lista de numeros entre 0-5 hay que sustituirlos por los simbolos
            elif mode == 'LFSR':
                nbits, tapindex, seed = 12, (12,11,10,4,1), 0b11001001
                #nbits, tapindex, seed = 8, (8,6,5,4,1), 0b11001001
                lfsrlist = []
                for xor, sr in lfsr2(seed, tapindex, nbits):
                    lfsr_gen = int(bin(2**nbits+sr)[3:], base=2)
                    lfsrlist.append(lfsr_gen % 5)
                self.seqIndex = 0 if self.seqIndex == len(lfsrlist)-1 else self.seqIndex + 1
                return LisAct[lfsrlist[self.seqIndex]]

            else:
                print('Error: random mode does not exist!')
                
        # value mode is PRNG or SEQ
        p2Move = genOneMove(self, self.p2Mode, stage)			# play one move from player2
        
        self.p2Count[dictAction[p2Move]] += 1
        p1Move = action
        self.p1Count[dictAction[p1Move]] += 1

        # check who won, set flag and assign reward 
        win, tie, lost = 0, 0, 0
        if MOVES_AND_REWARDS[p1Move, p2Move] == 0:
            self.cumTieCount, tie   = self.cumTieCount  + 1, 1
        elif MOVES_AND_REWARDS[p1Move, p2Move] == 1:
            self.cumWinCount, win   = self.cumWinCount  + 1, 1
        else:
            self.cumLostCount, lost = self.cumLostCount + 1, 1

        # update the running rates 
        self.cumWinRate = self.cumWinCount / moveCount
        self.cumTieRate = self.cumTieCount / moveCount
        self.cumLostRate = self.cumLostCount / moveCount
        # update moving avg buffer
        self.winRateBuf.append(self.cumWinRate) 
        self.tieRateBuf.append(self.cumTieRate)
        self.lostRateBuf.append(self.cumLostRate)
        # calculate trend
        tmp = [0, 0, 0]
        self.winRateTrend, self.tieRateTrend, self.lostRateTrend = 0, 0, 0
        if moveCount >= self.window:
            tmp[0] = sum(self.winRateBuf[i] for i in range(self.window)) / self.window
            tmp[1] = sum(self.tieRateBuf[i] for i in range(self.window)) / self.window
            tmp[2] = sum(self.lostRateBuf[i] for i in range(self.window)) / self.window
            # win rate trend analysis
            if self.winRateMovingAvg  < tmp[0]: 
                self.winRateTrend = 1		# win rate trending up. That's good
            else: 
                self.winRateTrend = 0		# win rate trending down. That's bad
            # tie rate trend analysis
            if self.tieRateMovingAvg  < tmp[1]:
                self.tieRateTrend = 1  		# tie rate trending up. That's bad
            else:
                self.tieRateTrend = 0  		# tie rate trending down.  Neutral
            # lost rate trend analysis
            if self.lostRateMovingAvg  < tmp[2]:
                self.lostRateTrend = 1  	# lst rate trending up.  That's bad
            else:
                self.lostRateTrend = 0  	# lost rate trending down. That's good
            self.winRateMovingAvg, self.tieRateMovingAvg, self.lostRateMovingAvg = tmp[0], tmp[1], tmp[2]
        # net reward in this round
        reward = win 									
        # record the state and reshape it for Keras input format
        dim = self.state.shape[1]
        self.state = np.array([\
            win, tie, lost, \
            self.winRateTrend, self.tieRateTrend, self.lostRateTrend, \
            self.winRateMovingAvg, self.tieRateMovingAvg, self.lostRateMovingAvg \
            ]).reshape(1, dim)
        # this game is done when it hits this goal
        done = False 
        return self.state, reward, done, dim


# ------------------------- class for the Double-DQN agent ---------------------------------
# facilities utilized here:
# 1)  Double DQN networks: one for behavior policy, one for target policy
# 2)  Learn from  sample from pool of memories 
# 3)  Basic TD-Learning stuff:  learning rate,  gamma for discounting future rewards
# 4)  Use of epsilon-greedy policy for controlling exploration vs exploitation
#-------------------------------------------------------------------------------------------
class NeuralHive(Agent):
    
    def __init__(self):
        super().__init__("NeuralHive")

        if saved_path:
            with open(f"{saved_path}/class.pkl", "rb") as saved_atrb:
                atrb = pickle.load(saved_atrb)
        self.env = RPSenv() if not saved_path else atrb["env"]
        self.action_space = self.env.action_space if not saved_path else atrb["action_space"]
        self.state = self.env.state if not saved_path else atrb["state"]
        # initialize the memory and auto drop when memory exceeds maxlen
        # this controls how far out in history the "expeience replay" can select from
        self.maxlen = 3000 if not saved_path else atrb["maxlen"]
        self.memory  = deque(maxlen = self.maxlen) if not saved_path else atrb["memory"]
        # future reward discount rate of the max Q of next state
        self.gamma = 0.9 if not saved_path else atrb["gamma"]
        # epsilon denotes the fraction of time dedicated to exploration (as oppse to exploitation)
        self.epsilon = 1.0 if not saved_path else atrb["epsilon"]
        self.epsilon_min = 0.01 if not saved_path else atrb["epsilon_min"]
        self.epsilon_decay = 0.9910 if not saved_path else atrb["epsilon_decay"]
        # model learning rate (use in backprop SGD process)
        self.learning_rate = 0.005 if not saved_path else atrb["learning_rate"]
        # transfer learning proportion contrl between the target and action/behavioral NN
        self.tau = .125 if not saved_path else atrb["tau"]
        # hyyperparameters for LSTM
        self.lookback = 100 if not saved_path else atrb["lookback"]
        self.hiddenUnits = 50 if not saved_path else atrb["hiddenUnits"]
        # create two models for double-DQN implementation
        self.model        = self.create_model() if not saved_path else load_model(f"{saved_path}/Model_DQN_NeuralHive.h5")
        self.target_model = self.create_model() if not saved_path else load_model(f"{saved_path}/TargetModel_DQN_NeuralHive.h5")
        # some space to collect TD target for instrumentaion
        self.TDtarget = [] if not saved_path else atrb["TDtarget"]
        self.TDtargetdelta = [] if not saved_path else atrb["TDtargetdelta"]
        self.Qmax =[] if not saved_path else atrb["Qmax"]
        # lookback steps accumlated
        self.step = len(self.memory)
    
    def save(self, dir_path):
        makedirs(dir_path, exist_ok=True)

        atrb = {
            "env": self.env,
            "action_space": self.action_space,
            "state": self.state,
            "maxlen": self.maxlen,
            "memory": self.memory,
            "gamma": self.gamma,
            "epsilon": self.epsilon,
            "epsilon_min": self.epsilon_min,
            "epsilon_decay": self.epsilon_decay,
            "learning_rate": self.learning_rate,
            "tau": self.tau,
            "lookback": self.lookback,
            "hiddenUnits": self.hiddenUnits,
            "TDtarget": self.TDtarget,
            "TDtargetdelta": self.TDtargetdelta,
            "Qmax": self.Qmax
        }
        with open(f"{dir_path}/class.pkl", "wb") as save_atrb:
            pickle.dump(atrb, save_atrb)
        
        self.model.save(f"{dir_path}/Model_DQN_NeuralHive.h5", save_format="h5")
        self.target_model.save(f"{dir_path}/TargetModel_DQN_NeuralHive.h5", save_format="h5")

    def decide(self, state):
        # AI agent take one action
        return self.act(state, self.step)

    def update(self, transition: Transition):
        self.step += 1
        # record the play into memory pool
        self.remember(transition.prev_state, transition.action, transition.reward, transition.next_state, False)
        # perform Q-learning from using |"experience replay": learn from random samples in memory
        self.replay()
        # apply tranfer learning from actions model to the target model.
        self.target_train()

    def create_model(self):
        input_feature_dim = self.state.shape[1]
        output_feature_dim = len(self.action_space)
        model = Sequential()
        #model.add(GRU(self.hiddenUnits,\
        model.add(LSTM(self.hiddenUnits,\
		    return_sequences = False,\
		    #activation = None, \
		    #recurrent_activation = None, \
            input_shape = (self.lookback, input_feature_dim)))
		# let the output be the predicted target value.  NOTE: do not use activation to squash it! 
        #model.add(TimeDistributed(Dense(4)))
        #model.add(Flatten())
        model.add(Dense(output_feature_dim))
        model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=self.learning_rate))
        print(model.summary())
        return model

    def act(self, state, step):
    	# this is to take one action
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        # decide to take a random exploration or make a policy-based action (thru NN prediction)
        # with a LSTM design, delay on policy prediction after at least lookback steps have accumlated
        if np.random.random() < self.epsilon or step < self.lookback + 1:
        	# return a random move from action space
            return random.choice(list(self.action_space))
        else:
            # return a policy move
            state_set = np.empty((1, self.state.shape[1]))  # iniitial with 2 dims
            for j in range(self.lookback):
                state_tmp, _, _, _, _  = self.memory[-(j+1)]    # get the most recent state and the previous N states
                if j == 0:
                    state_set[0] = state_tmp                 	# iniitalize the first record
                else:
                    state_set = np.concatenate((state_set, state_tmp), axis = 0)		# get a consecutive set of states for LSTM prediction        
            state_set = state_set[None, :, :]  					# make the tensor 3 dim to align with Keras reqmt
            #print(state_set)
            #print(state_set.shape)
            self.Qmax.append(max(self.model.predict(state_set, verbose=0, workers=cpu_count()*0.75)[0]))
            return list(Action)[np.argmax(self.model.predict(state_set, verbose=0, workers=cpu_count()*0.75)[0])]

    def remember(self, state, action, reward, new_state, done):
		# store up a big pool of memory
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):  		
    	# DeepMind "experience replay" method
    	# do the training (learning); this is DeepMind tricks of using "Double" model (Mnih 2015)
    	# the sample size from memory to learn from
     	#------------------------
        # do nothing untl the memory is large enough
        RL_batch_size = 24  # this is experience replay batch_size (not the LSTM fitting batch size)
        if len(self.memory) < RL_batch_size: return
        # get the samples; each sample is a sequence of consecutive states with same lookback length as LSTM definition
        for i in range(RL_batch_size):
            state_set     = np.empty((1, self.state.shape[1]))
            new_state_set = np.empty((1, self.state.shape[1]))
            if len(self.memory) <= self.lookback:							# check if memory is large enough to retrieve the time sequence
                return
            else:
                a = random.randint(0, len(self.memory) - self.lookback)		# first get a random location
            state, action, reward, new_state, done = self.memory[-(a+1)]    # retrieve a sample from memory at that loc; latest element at the end of deque
            for j in range(self.lookback):
                state_tmp, _, _, new_state_tmp, _  = self.memory[-(a+j+1)]  # get a consecutive set of states
                if j == 0:
                    state_set[0] = state_tmp
                    new_state_set[0] = new_state_tmp
                else:
                    state_set = np.concatenate((state_set, state_tmp), axis = 0)		# get a consecutive set of states for LSTM prediction
                    new_state_set = np.concatenate((new_state_set, new_state_tmp), axis = 0)  
            # do the prediction from current state
            state_set     = state_set[None, :, :]							# make the tensor 3 dim to align with Keras reqmt
            new_state_set = new_state_set[None, :, :]						# make the tensor 3 dim to align with Keras reqmt
            target        = self.target_model.predict(state_set, verbose=0, workers=cpu_count()*0.75)
            # do the Q leanring
            if done:
                target[0][dictAction[action]] = reward
            else:
                Q_future = max(self.target_model.predict(new_state_set, verbose=0, workers=cpu_count()*0.75)[0]) 
                TDtarget = reward + Q_future * self.gamma
                self.TDtarget.append(TDtarget)
                self.TDtargetdelta.append(TDtarget - target[0][dictAction[action]])
                target[0][dictAction[action]] = TDtarget	 			
            # do one pass gradient descend using target as 'label' to train the action model
            self.model.fit(state_set, target, batch_size = 1,  epochs = 1, verbose = 0, workers=cpu_count()*0.75)
        
    def target_train(self):
    	# transfer weights  proportionally from the action/behave model to the target model
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)


2023-06-01 18:58:08.278049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 18:58:08.478415: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 18:58:08.558641: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 18:58:09.220198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

## Competición

In [7]:
def competition(competitors, friendly_sets, competition_sets):
    leaderboard = {c: 0 for c in competitors}
    game = Game()
    for p1, p2 in itertools.combinations(leaderboard.keys(), 2):
        p1, p2 = p1(), p2()
        
        # Amistoso
        s = 0
        for i in range(friendly_sets):
            a1 = p1.decide(s)
            a2 = p2.decide(s)
            reward = game.play(a1, a2)
            p1.update(Transition(prev_state=s, next_state=s, action=a1, reward=reward))
            p2.update(Transition(prev_state=s, next_state=s, action=a2, reward=-reward))

         # Competición
        s = 0
        r1 = r2 = 0
        for i in range(competition_sets):
            a1 = p1.decide(s)
            a2 = p2.decide(s)
            reward = game.play(a1, a2)
            r1 += reward
            r2 -= reward

        # Actualización de marcadores globales
        if r1 > r2:
            leaderboard[p1.__class__] += 3
        elif r2 > r1:
            leaderboard[p2.__class__] += 3
        else:
            leaderboard[p1.__class__] += 1
            leaderboard[p2.__class__] += 1
    
    return leaderboard

In [8]:
print('GLOBAL LEADERBOARD')

NUM_MATCHES = 100       # Número de torneos (para que sea más equitativo)
FRIENDLY_SETS = 10000   # Número de partidas amistosas (para aprender)
COMPETITION_SETS = 100  # Número de partidas de competición (para ganar)

# Los competidores de los grupos. Se han comentado los que no cumplen las
# condiciones que se indicaban en el enunciado.
competitors = [
    # AiWreck,
    # AiGigantix, # No válido
    # Musasher,     
    # DeepDark,
    # KobayashiMaru,
    # NetCoreAI,
    NeuralHive,
    NeuralHive,
    # Neuronetix, # No válido
    # Rosmi,
    # BAITbot,
    # LizardWizardOnTheBlizzard,
    # TechNoirAgent,
]

# Se ejecutan todos los torneos consecutivamente, guardando cada marcador
leaderboards = []
for i in range(COMPETITION_SETS):
    print(f'.', end='')
    leaderboard = competition(competitors, FRIENDLY_SETS, COMPETITION_SETS)
    leaderboards.append(leaderboard)
print('🏁')

# Obtenemos el marcador global haciendo la media enrte todos los torneos
gl = {
    c: sum(lb[c] for lb in leaderboards) / COMPETITION_SETS
    for c in competitors
}
print('-' * 80)
for k, v in sorted(gl.items(), key=lambda t: t[1], reverse=True):
     print(f'{v:<10}\t{k()}')

GLOBAL LEADERBOARD
....................................................................................................🏁
--------------------------------------------------------------------------------


2023-06-01 18:58:41.971199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 18:58:41.993211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 18:58:41.993420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 18:58:41.994258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

0.0       	NeuralHive


***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>